# Data Mining & Machine Learning Project - Group Nvidia
## By Adrian Sestan, Hugo Macedo & Igor Ranisavljevic

In [59]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
import spacy
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import re, string, unicodedata
import gensim
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn import datasets
from sklearn.utils.multiclass import unique_labels
import numpy as np
import matplotlib.pyplot as plt

In [60]:
path = "https://raw.githubusercontent.com/TetraFaal/DMML-Project-Nvidia/master/data/82653de5-2ed1-4782-959e-23eba75d67a9_training_data.csv"
df = pd.read_csv(path)
df.keyword.fillna('#', inplace=True)
df['x'] = df['keyword']+ ' ' + df['text'] 
df.drop('text',axis='columns', inplace=True)
df = df.rename({'x': 'text'}, axis=1)
df['text'] = df['text'].str.replace('nan', "")
df


,id,keyword,location,target,text
0,3738,destroyed,USA,0,destroyed Black Eye 9: A space battle occurred...
1,853,bioterror,NaN,0,bioterror #world FedEx no longer to transport ...
2,10540,windstorm,"Palm Beach County, FL",1,windstorm Reality Training: Train falls off el...
3,5988,hazardous,USA,1,hazardous #Taiwan Grace: expect that large roc...
4,6328,hostage,Australia,1,hostage New ISIS Video: ISIS Threatens to Behe...
...,...,...,...,...,...
6466,4377,earthquake,ARGENTINA,1,earthquake #Earthquake #Sismo M 1.9 - 15km E o...
6467,3408,derail,NaN,0,derail @EmiiliexIrwin Totally agree.She is 23 ...
6468,9794,trapped,NaN,1,trapped Hollywood Movie About Trapped Miners R...
6469,10344,weapons,Beirut/Toronto,1,weapons Friendly reminder that the only countr...


In [61]:
path_test = "https://raw.githubusercontent.com/TetraFaal/DMML-Project-Nvidia/master/data/5fc21f33-b209-4b07-ad70-a69020dfd2cf_test_data.csv"
df_test = pd.read_csv(path_test)
df_test.keyword.fillna('#', inplace=True)
df_test['x'] = df_test['keyword']+ ' ' + df_test['text'] 
df_test.drop('text',axis='columns', inplace=True)
df_test = df_test.rename({'x': 'text'}, axis=1) 
df_test['text'] = df_test['text'].str.replace('nan', "")
df_test['text']

0       tsunami Crptotech tsunami and banks.\n http://...
1       traumatised I'm that traumatised that I can't ...
2       burning%20buildings @foxnewsvideo @AIIAmerican...
3       desolate Me watching Law &amp; Order (IB: @sau...
4               crushed Papi absolutely crushed that ball
                              ...                        
1137    derailed @ItsQueenBaby I'm at work it's a bunc...
1138    suicide%20bomber #?? #?? #??? #??? Suicide bom...
1139    volcano Eruption of Indonesian volcano sparks ...
1140    fear Never let fear get in the way of achievin...
1141    refugees wowo--=== 12000 Nigerian refugees rep...
Name: text, Length: 1142, dtype: object

In [62]:
display(df.shape, df.info(), df.describe, df.keyword.value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6471 entries, 0 to 6470
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        6471 non-null   int64 
 1   keyword   6471 non-null   object
 2   location  4330 non-null   object
 3   target    6471 non-null   int64 
 4   text      6471 non-null   object
dtypes: int64(2), object(3)
memory usage: 252.9+ KB


(6471, 5)

None

<bound method NDFrame.describe of          id  ...                                               text
0      3738  ...  destroyed Black Eye 9: A space battle occurred...
1       853  ...  bioterror #world FedEx no longer to transport ...
2     10540  ...  windstorm Reality Training: Train falls off el...
3      5988  ...  hazardous #Taiwan Grace: expect that large roc...
4      6328  ...  hostage New ISIS Video: ISIS Threatens to Behe...
...     ...  ...                                                ...
6466   4377  ...  earthquake #Earthquake #Sismo M 1.9 - 15km E o...
6467   3408  ...  derail @EmiiliexIrwin Totally agree.She is 23 ...
6468   9794  ...  trapped Hollywood Movie About Trapped Miners R...
6469  10344  ...  weapons Friendly reminder that the only countr...
6470   1779  ...  buildings%20on%20fire Buildings are on fire an...

[6471 rows x 5 columns]>

#                        55
deluge                   39
earthquake               38
collision                37
harm                     37
                         ..
demolished               18
epicentre                10
threat                   10
radiation%20emergency     9
inundation                6
Name: keyword, Length: 222, dtype: int64

In [63]:
max(len(df[df["target"] == 1]) / len(df), len(df[df["target"] == 0]) / len(df))

0.5719363313243703

In [64]:
sp = spacy.load('en_core_web_sm')
import re
# Create tokenizer function
def spacy_tokenizer(text):

    # Define stopwords, punctuation, and numbers
    stop_words = spacy.lang.en.stop_words.STOP_WORDS
    punctuations = string.punctuation
    numbers = "0123456789"
    mentions = "(@[A-Za-z0–9]+)|([⁰-9A-Za-z \t])|(\w+:\/\/\S+)" 

    # Create spacy object
    mytokens = sp(text)

    # Lemmatize each token and convert each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Remove stop words and punctuation
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]
    mytokens = [ word for word in mytokens if word not in mentions ]

    mytokens = [ word.replace('xx/', '').replace('xxxx/', '').replace('xx', '') for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in ["xxxx", "xx", ""] ]

      # Remove sufix like ".[1" in "experience.[1"
    mytokens_2 = []
    for word in mytokens:
      for char in word:
        if (char in punctuations) or (char in numbers):
          word = word.replace(char, "")
      if word != "":
        mytokens_2.append(word)

    # Return preprocessed list of tokens
    return mytokens_2


In [65]:
sp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS
punctuations = string.punctuation
numbers = "0123456789"
mentions = "(@[A-Za-z0–9]+)|([⁰-9A-Za-z \t])|(\w+:\/\/\S+)" 
print('sw ',stop_words)
print('punctuation ', punctuations)
print(numbers)
print(mentions)

sw  {'her', 'then', 'those', 'meanwhile', 'thereupon', '‘re', 'above', 'yourself', "n't", 'whenever', 'anyway', 'many', 'due', 'such', 'fifteen', 'either', 'latter', 'quite', 'regarding', 'so', 'for', 'upon', 'ours', 'four', 'bottom', 'whose', 'becoming', 'during', 'no', 'used', 'with', '‘ll', 'were', 'it', 'some', 'twenty', 'someone', 'ca', 'various', 'themselves', 'nor', 'until', 'whereafter', 'much', '’m', 'part', 'never', 'you', 'beforehand', 'rather', 'is', 'that', 'whereby', 'be', 'yourselves', 'but', 'has', 'behind', 'she', 'does', 'sixty', 'from', 'at', 'me', 'name', 'must', 'thence', 'would', '’s', 'without', 'thru', 'per', 'up', 'elsewhere', 'do', 'beside', 'beyond', 'herself', 'should', 'down', 'whither', 'unless', 'anyone', 'therein', 'further', 'along', 'anyhow', 'both', 'six', '’re', 'mostly', 'noone', 'somewhere', 'within', 'thereafter', "'s", 'also', 'thus', 'another', 'front', 'afterwards', 'too', 'when', 'whoever', 'sometime', 'might', 'nowhere', 'nine', 'being', 'mor

In [66]:
#train/test 

X_train = df["text"]
X_test = df_test["text"]
y_train = df['target']



display((X_train,y_train))


(0       destroyed Black Eye 9: A space battle occurred...
 1       bioterror #world FedEx no longer to transport ...
 2       windstorm Reality Training: Train falls off el...
 3       hazardous #Taiwan Grace: expect that large roc...
 4       hostage New ISIS Video: ISIS Threatens to Behe...
                               ...                        
 6466    earthquake #Earthquake #Sismo M 1.9 - 15km E o...
 6467    derail @EmiiliexIrwin Totally agree.She is 23 ...
 6468    trapped Hollywood Movie About Trapped Miners R...
 6469    weapons Friendly reminder that the only countr...
 6470    buildings%20on%20fire Buildings are on fire an...
 Name: text, Length: 6471, dtype: object, 0       0
 1       0
 2       1
 3       1
 4       1
        ..
 6466    1
 6467    0
 6468    1
 6469    1
 6470    1
 Name: target, Length: 6471, dtype: int64)

In [67]:
tfidf_vector = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), tokenizer=spacy_tokenizer)

In [68]:
df_test

,id,keyword,location,text
0,9972,tsunami,NaN,tsunami Crptotech tsunami and banks.\n http://...
1,9865,traumatised,"Portsmouth, UK",traumatised I'm that traumatised that I can't ...
2,1937,burning%20buildings,NaN,burning%20buildings @foxnewsvideo @AIIAmerican...
3,3560,desolate,NaN,desolate Me watching Law &amp; Order (IB: @sau...
4,2731,crushed,bahstun/porta reeko,crushed Papi absolutely crushed that ball
...,...,...,...,...
1137,3497,derailed,NaN,derailed @ItsQueenBaby I'm at work it's a bunc...
1138,9191,suicide%20bomber,NaN,suicide%20bomber #?? #?? #??? #??? Suicide bom...
1139,10217,volcano,NaN,volcano Eruption of Indonesian volcano sparks ...
1140,5308,fear,New York,fear Never let fear get in the way of achievin...


In [69]:
X_test

0       tsunami Crptotech tsunami and banks.\n http://...
1       traumatised I'm that traumatised that I can't ...
2       burning%20buildings @foxnewsvideo @AIIAmerican...
3       desolate Me watching Law &amp; Order (IB: @sau...
4               crushed Papi absolutely crushed that ball
                              ...                        
1137    derailed @ItsQueenBaby I'm at work it's a bunc...
1138    suicide%20bomber #?? #?? #??? #??? Suicide bom...
1139    volcano Eruption of Indonesian volcano sparks ...
1140    fear Never let fear get in the way of achievin...
1141    refugees wowo--=== 12000 Nigerian refugees rep...
Name: text, Length: 1142, dtype: object

In [70]:
# Define classifier
classifier = LogisticRegression(C=1.2067926406393288, max_iter=5000, n_jobs=2,
                   random_state=72)

# Create pipeline
pipe = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', classifier)])
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
y_pred_s = pipe.predict(X_train)



In [71]:
y_pred.shape

(1142,)

In [72]:
print(round(accuracy_score(y_train, y_pred_s), 4))

0.8583


In [73]:
test_text = pd.DataFrame(y_pred)
test_text.to_csv('test_text.csv' , index = False)